In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:24pt;}
div.text_cell_render.rendered_html{font-size:20pt;}
div.text_cell_render li, div.text_cell_render p, code{font-size:22pt; line-height:40px;}
div.output {font-size:24pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:24pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:24pt;padding:5px;}
table.dataframe{font-size:24px;}
</style>
"""))

# 벡터DB : Chroma vs. Pinecone
- Chroma : 인메모리 vector DB, 로컬 vector DB
- Pinecone : 클라우드 vector DB
    (https://www.pinecone.io에서 api key 생성 -> .env에 추가(PINECONE_API_KEY등록)

# 0. 패키지 설치

In [2]:
%pip install -q pinecone langchain-pinecone --no-warn-script-location

Note: you may need to restart the kernel to use updated packages.


# 1. knowledge Base 구성을 위한 데이터 생성

In [3]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
loader = Docx2txtLoader('data/소득세법(법률)(제21065호)(20260102).docx')
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, 
    chunk_overlap=200,
    # separators=["\n\n", "\n", " ", ""]
)
document_list = loader.load_and_split(text_splitter=text_splitter)
len(document_list)

193

In [4]:
# embedding : upstage의 solar-embedding-1-large-passage
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings
load_dotenv()
embedding = UpstageEmbeddings(model="solar-embedding-1-large-passage")

In [5]:
len(embedding.embed_query("소득세법"))

4096

In [6]:
%%time
# pinecone vector database
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
import os
pc = Pinecone(
    api_key=os.getenv("PINECONE_API_KEY")
)
# 데이터를 처음 업로드할 때
index_name = "tax-index-upstage"
# database = PineconeVectorStore.from_documents(
#     documents=document_list,
#     embedding=embedding,
#     index_name=index_name
# )
# 업로드시 경고가 안보이려면 아나콘다 프롬프트 llm환경에서 conda install -c conda-forge ipywidgets

# 업로드한 벡터db를 가져올 때
database = PineconeVectorStore(
    embedding=embedding,# 질문을 임베딩하여 유사도 검색
    index_name=index_name
)

CPU times: total: 328 ms
Wall time: 1.93 s


# 2. 답변 생성을 위한 Retrieval

In [7]:
query = "연봉이 5천만원인 직장인의 소득세는 얼마인가요?"
retrieved_docs = database.similarity_search(query, k=3) # 기본k값:4

In [8]:
# retrieved_docs[2].page_content
retrieved_doc = "\n\n---\n\n".join([doc.page_content for doc in retrieved_docs])

In [9]:
retriever = database.as_retriever(
    search_kwargs={"k":3}
)
retrived_docs = retriever.invoke(query)
retrieved_doc = "\n\n---\n\n".join([doc.page_content for doc in retrieved_docs])

# 3. 답변 생성

In [10]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model = "gpt-4.1-nano")

In [11]:
# upstage에서 받은 20$로 llm을 사용하고 싶다면
from langchain_upstage import ChatUpstage
llm = ChatUpstage(
    model = "solar-pro2",
    reasoning_effort="high" #느리지만 더 깊게 추론함 (low, medium)
)

In [12]:
prompt = f"""[identity]
- 당신은 최고의 한국 소득세법 전문가입니다
- [context]를 참고해서 사용자의 질문에 답변해 주세요.
- [context]는 다음과 같아요
{retrieved_doc}
- 질문 : {query}"""

In [13]:
ai_message = llm.invoke(prompt)

In [14]:
print(ai_message.content)

**연봉 5천만원 직장인의 소득세 계산 (2023년 기준)**

### 1. **근로소득금액**  
- 연봉 5,000만원 = **5,000만원** (근로소득 전액)

---

### 2. **근로소득 공제**  
- **5,000만원 이하** 구간 적용:  
  \[
  1,200만원 + (5,000만원 - 1,200만원) \times 15\% = 1,200만원 + 540만원 = **1,740만원**
  \]  
- **과세표준**:  
  \[
  5,000만원 - 1,740만원 = **3,260만원**
  \]

---

### 3. **종합소득세 계산**  
- **세율 구간**: 1,200만원 초과 ~ 4,600만원 이하  
  - 1,200만원 × 6% = **72만원**  
  - (3,260만원 - 1,200만원) × 15% = **2,060만원 × 15% = 309만원**  
  - **산출세액**: 72만원 + 309만원 = **381만원**  
- **누진공제**: 108만원  
  \[
  381만원 - 108만원 = **273만원**
  \]

---

### 4. **지방소득세**  
- 종합소득세의 10%:  
  \[
  273만원 × 10\% = **27.3만원**
  \]

---

### 5. **총 납부 세액**  
\[
273만원 (종합소득세) + 27.3만원 (지방소득세) = **273만원 + 27.3만원 = 300.3만원**
\]

---

### **최종 답변**  
- **예상 소득세**: 약 **300만원**  
- **참고**: 실제 세금은 의료비, 보험료, 신용카드 소득공제 등 추가 공제에 따라 달라질 수 있습니다. 정확한 계산을 위해선 [홈택스](https://www.hometax.go.kr)에서 연말정산 시뮬레이션을 활용하시기 바랍니다.


# 4. langchain 답변 생성
- ch9.07_LangChain과 vectorDatabase을 활용한 RAG구현(UpstageEmbedding) 참조